In [3]:
import pandas as pd
import requests
import datetime as dt
import json
import sys
import os

In [6]:
##get latest trailhead hotspot dataset

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots/trailheadHotspots_Y2022_M1_D11.csv')
trailheadHotspots.head(5)

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName
0,L10128988,Berntsen Park,US,US-WA,US-WA-033,47.538439,-122.043281,2021-12-15 11:46,15.0,EastSunsetWay
1,L10129014,"Confluence Park, Issaquah",US,US-WA,US-WA-033,47.536469,-122.039342,2022-01-08 12:54,44.0,EastSunsetWay
2,L8365620,Issaquah High School,US,US-WA,US-WA-033,47.522490,-122.028687,2021-12-17 08:00,58.0,EastSunsetWay
3,L8102503,Issaquah Salmon Hatchery,US,US-WA,US-WA-033,47.529503,-122.039512,2021-12-03 08:22,33.0,EastSunsetWay
4,L3352189,"Maple Street ponds, Issaquah",US,US-WA,US-WA-033,47.543027,-122.053715,2021-11-30 10:12,46.0,EastSunsetWay


In [27]:
##pull in trailhead reference as base for duplicate hotspot removal
trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())

trailheadRef.head()

,Route,StopName,Address,Latitude,Longitude
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
4,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.506620,-122.086660


In [10]:
trailheadHotspots['StopName'].value_counts()

DiscoveryParkSouth          26
DiscoveryParkNorth          22
EastSunsetWay               14
HighSchool                  14
MargaretsWay                10
ChiricoTrail_PooPooPoint     7
SquakMountain                6
MountTeneriffe               6
MountSi                      5
LittleSi                     1
Name: StopName, dtype: int64

In [25]:
##filter to Discovery Park for testing purposes
DiscTH_hotspots = pd.DataFrame()

DiscFilter = ['DiscoveryParkSouth','DiscoveryParkNorth']
DiscTH_hotspots = trailheadHotspots[trailheadHotspots['StopName'].isin(DiscFilter)]

DiscTH_hotspots.head()

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName
63,L5980530,11th Ave NW Street End Park (Ballard),US,US-WA,US-WA-033,47.659750,-122.370775,2021-01-13 12:20,33.0,DiscoveryParkNorth
64,L1096378,14th Ave NW Boat Ramp (Ballard),US,US-WA,US-WA-033,47.660798,-122.373628,2021-12-19 14:45,53.0,DiscoveryParkNorth
65,L6169475,"32nd Ave W Boat Launch, Magnolia",US,US-WA,US-WA-033,47.632088,-122.398792,2021-12-19 10:49,99.0,DiscoveryParkNorth
66,L5395316,Commodore Park,US,US-WA,US-WA-033,47.665693,-122.400965,2022-01-08 07:36,94.0,DiscoveryParkNorth
67,L128530,Discovery Park,US,US-WA,US-WA-033,47.663868,-122.421532,2022-01-09 14:21,287.0,DiscoveryParkNorth


In [93]:
##series of trailheads with duplicate locations
DiscDup_filter = pd.DataFrame()
DiscDup_filter = DiscTH_hotspots['locName'].value_counts

##merge series with trailhead_ref


In [63]:
dfk = pd.merge(
    left=DiscTH_hotspots,
    right=trailheadRef,
    how='left',
    left_on='StopName',
    right_on='StopName')

In [77]:
##calculate absolute distance from stop location
for i in dfk.itertuples():
    latInt = dfk['lat']
    latitudeInt = dfk['Latitude']
    lngInt = dfk['lng']
    longitudeInt = dfk['Longitude']
    dfk['distanceLatitude'] = abs(latitudeInt-latInt)
    dfk['distanceLongitude'] = abs(longitudeInt-lngInt)
dfk['distanceLatitude'].astype('int32')

dfk.head()

,locId,locName,countryCode,subnational1Code,subnational2Code,lat,lng,latestObsDt,numSpeciesAllTime,StopName,Route,Address,Latitude,Longitude,distanceLatitude,distanceLongitude,duplicates
0,L5980530,11th Ave NW Street End Park (Ballard),US,US-WA,US-WA-033,47.659750,-122.370775,2021-01-13 12:20,33.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.004881,0.040288,NaN
1,L1096378,14th Ave NW Boat Ramp (Ballard),US,US-WA,US-WA-033,47.660798,-122.373628,2021-12-19 14:45,53.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.003833,0.037435,NaN
2,L6169475,"32nd Ave W Boat Launch, Magnolia",US,US-WA,US-WA-033,47.632088,-122.398792,2021-12-19 10:49,99.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.032543,0.012271,NaN
3,L5395316,Commodore Park,US,US-WA,US-WA-033,47.665693,-122.400965,2022-01-08 07:36,94.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.001062,0.010098,NaN
4,L128530,Discovery Park,US,US-WA,US-WA-033,47.663868,-122.421532,2022-01-09 14:21,287.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.000763,0.010469,NaN
5,L1924233,Discovery Park--South Meadow,US,US-WA,US-WA-033,47.655213,-122.414131,2022-01-09 09:48,165.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.009418,0.003068,NaN
6,L3440023,Discovery Park--Visitor Center,US,US-WA,US-WA-033,47.658152,-122.406103,2022-01-09 07:54,128.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.006479,0.004960,NaN
7,L571490,Discovery Park--West Point,US,US-WA,US-WA-033,47.661870,-122.435530,2022-01-09 14:59,214.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.002761,0.024467,NaN
8,L1465184,Fisherman's Terminal (Magnolia),US,US-WA,US-WA-033,47.655874,-122.380624,2022-01-09 10:43,61.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.008757,0.030439,NaN
9,L7906413,"Four Mile Rock, Magnolia",US,US-WA,US-WA-033,47.638688,-122.412486,2021-12-12 10:49,116.0,DiscoveryParkNorth,SeattleParks,"Discovery Park Road & Texas Way, Seattle, WA 9...",47.664631,-122.411063,0.025943,0.001423,NaN
